In [1]:
from pandas import read_csv
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = read_csv('data/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Testing framework

The framework is to train a linear and linear with L2 regularisation on a single feature and observe the MAE, RMSE, R2 scores. Then I will repeat this for each feature to determine which one provides a better model based on the statistical tests.


In [3]:
from sklearn.linear_model import LinearRegression, Ridge
from pandas import get_dummies
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

In [4]:
def test_feature(x, y):
    x = x.values.reshape(-1, 1)
    y = y.values.reshape(-1)
    models = [('Linear', LinearRegression()), ('L2 linear',Ridge())]
    for mx in models:
        clf = mx[1]
        clf.fit(x, y)
        y_ = clf.predict(x)
        print(mx[0])
        print('MAE: %s' % mean_absolute_error(y, y_))
        print('RMSE: %s' %sqrt(mean_squared_error(y, y_)))
        print('R2: %s' %r2_score(y, y_))
        print()
    
def test_feature_cat(x, y):
    x = get_dummies(x).replace({False:0, True:1}).values
    y = y.values.reshape(-1)
    models = [('Linear', LinearRegression()), ('L2 linear',Ridge())]
    for mx in models:
        clf = mx[1]
        clf.fit(x, y)
        y_ = clf.predict(x)
        print(mx[0])
        print('MAE: %s' % mean_absolute_error(y, y_))
        print('RMSE: %s' %sqrt(mean_squared_error(y, y_)))
        print('R2: %s' %r2_score(y, y_))
        print()

## One-way ANOVA test

Each value of a categorical feature is a group and the continuous data is the independent variable (charges), the ANOVA test is performed to determine if there is a difference between the means of group formed from a unique categorical feature value based on the means between groups and variance within groups.

### H0: there is no difference in the mean between groups for a particular feature
### H1: the means are different in the mean between groups for a particular feature

Low P-values are strong evidence to reject the null hypothesis, and is an indication that the feature is valuable for model.

In [5]:
from scipy.stats import f_oneway

In [6]:
group_a = data[data['sex']=='male']['charges']
group_b = data[data['sex']=='female']['charges']
F_stat, p_val = f_oneway(group_a, group_b, axis=0)
print(F_stat)
print(p_val)

4.3997016974374565
0.03613272100596256


In [7]:
group_a = data[data['region']=='southwest']['charges']
group_b = data[data['region']=='southeast']['charges']
group_c = data[data['region']=='northwest']['charges']
group_d = data[data['region']=='northeast']['charges']
F_stat, p_val = f_oneway(group_a, group_b, axis=0)
print(F_stat)
print(p_val)

5.8960452705730475
0.015430651095692707


The p_values show us that region there is a slightly more significant difference between the means between groups for the feature 'region' than 'sex'.

In [8]:
# best feature
test_feature_cat(data['region'], data['charges'])

Linear
MAE: 9052.126035834242
RMSE: 12065.264162780724
R2: 0.006634016807031462

L2 linear
MAE: 9052.151966970734
RMSE: 12065.264493787838
R2: 0.006633962301600738



In [9]:
# worst feature
test_feature_cat(data['sex'], data['charges'])

Linear
MAE: 9108.376246233931
RMSE: 12085.932140833916
R2: 0.00322780026625491

L2 linear
MAE: 9088.672317992896
RMSE: 12085.60128674103
R2: 0.0032823730777093996



## Results

As expected, region is a slightly better performing input for a regression model than sex

# Pearson's R on polynomial model

In [13]:
from sklearn.feature_selection import r_regression, f_regression
from sklearn.preprocessing import PolynomialFeatures

In [11]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
x = data[['age', 'bmi','children']]
x_poly_deg2 = poly.fit_transform(x)
x_poly_features = ['age', 'bmi','children','age^2','age*bmi','age*children','bmi^2','bmi*children','children^2']

In [14]:
y = data['charges'].values
F_stat, p_val = f_regression(x_poly_deg2, y)
pear_r = r_regression(x_poly_deg2, y)
print(F_stat)
print(p_val)
R = []
for i in range(len(pear_r)):
    R.append((x_poly_features[i], pear_r[i]))
R.sort(key=lambda x:x[1])
for r in R:
    print(r)

[131.17401258  54.70930805   6.20603705 132.88063348 168.60647718
  23.67798385  51.67928279  14.21103762   2.00382014]
[4.88669333e-29 2.45908554e-13 1.28521285e-02 2.23481907e-29
 2.14730462e-36 1.27416258e-06 1.08396093e-12 1.70537427e-04
 1.57136396e-01]
('children^2', 0.038699084615013)
('children', 0.06799822684790474)
('bmi*children', 0.10259166251066097)
('age*children', 0.13196365243068287)
('bmi^2', 0.19298061523991067)
('bmi', 0.19834096883363017)
('age', 0.2990081933306476)
('age^2', 0.30077213074497)
('age*bmi', 0.33475391409782135)


In [15]:
from pandas import DataFrame
data_poly = DataFrame(data=x_poly_deg2, columns=x_poly_features)
data_poly

,age,bmi,children,age^2,age*bmi,age*children,bmi^2,bmi*children,children^2
0,19.0,27.900,0.0,361.0,530.100,0.0,778.410000,0.00,0.0
1,18.0,33.770,1.0,324.0,607.860,18.0,1140.412900,33.77,1.0
2,28.0,33.000,3.0,784.0,924.000,84.0,1089.000000,99.00,9.0
3,33.0,22.705,0.0,1089.0,749.265,0.0,515.517025,0.00,0.0
4,32.0,28.880,0.0,1024.0,924.160,0.0,834.054400,0.00,0.0
...,...,...,...,...,...,...,...,...,...
1333,50.0,30.970,3.0,2500.0,1548.500,150.0,959.140900,92.91,9.0
1334,18.0,31.920,0.0,324.0,574.560,0.0,1018.886400,0.00,0.0
1335,18.0,36.850,0.0,324.0,663.300,0.0,1357.922500,0.00,0.0
1336,21.0,25.800,0.0,441.0,541.800,0.0,665.640000,0.00,0.0


In [20]:
test_feature(data_poly['age*bmi'], data['charges'])

Linear
MAE: 9046.935685510143
RMSE: 11407.066111073244
R2: 0.1120601830038147

L2 linear
MAE: 9046.935685322564
RMSE: 11407.066111073244
R2: 0.1120601830038147



In [17]:
test_feature(data_poly['children'], data['charges'])

Linear
MAE: 9097.505408669633
RMSE: 12077.466128266162
R2: 0.004623758854459203

L2 linear
MAE: 9097.488724670367
RMSE: 12077.466135689268
R2: 0.004623757630894176



## Results

There is little modelling performance difference between the best and worst features as selected by Peason's R scores, even when using polynomial features. The order of the performance is correct, there are limitations to using Pearson's R for feature selection, perhaps other correlation scores such as spearman will perform better.